MIT License

Copyright (c) 2021 alxyok

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Solving the 3D Burger's equation

**FORGET ABOUT THIS CODE, IT'S UNTRACTABLE. FOR EDUCATIONAL PURPOSES ONLY.**

**Let's have some fun! Untested fun even 😛 (It works, but what it really does no one knows for certain!)**

**Most of the content below is largely inspired from the work of [Raissi et al.](https://maziarraissi.github.io/PINNs/) as weel as [Liu et al.](https://www.sciencedirect.com/science/article/abs/pii/S0142727X21000527), please refer to those papers for a comprehensive theoretical understanding.**

The Burger's equation is one of the well-studied fundamental PDEs in that exhibit shocks, and for which an non-trivial analytical solution exists in the Physics litterature. A conjunction of factors (profusion of data, capable cheap hardware, and backprop) has lead to the resurection Deep Learning (DL) which has in turn paved the way for the development of scientific machine learning libraries such as TensorFlow and PyTorch. 

Those frameworks come with free auto-differentiation, a key tool for this lab which will enable the development of a self-supervised neural model based on residuals.

We'll use PyTorch, but TensorFlow + Keras could do just as fine. Be sure to check out [PyTorch Tutorials](https://pytorch.org/tutorials/) and [PyTorch API](https://pytorch.org/docs/1.9.0/), which are a great source of information. Also, [Stackoverflow](https://stackoverflow.com/questions/tagged/pytorch).

In [34]:
import this

In [35]:
import matplotlib.pyplot as plt
import numpy as np

# we'll use PyTorch, but TensorFlow + Keras could do just as fine
import torch

from torch import nn 

## 1. Problem statement

Note: we do not use the Hopf-Cole transformation that would allow for a simplified formula but instead use the raw explicit formulation of the problem. 

We propose to solve the 3D nonlinear Burger's problem defined by the following set of equations:

$u_t + u * u_x + v * u_y + w * u_z - \frac{1}{R_e} (u_{xx} + u_{yy} + u_{zz}) = 0$

$v_t + u * v_x + v * v_y + w * v_z - \frac{1}{R_e} (v_{xx} + v_{yy} + v_{zz}) = 0$

$w_t + u * w_x + v * w_y + w * w_z - \frac{1}{R_e} (w_{xx} + w_{yy} + w_{zz}) = 0$

in which $Re$ is the Reynolds number, which characterizes the fluid flow behavior in various situations, and under the initial condition and boundary conditions defined below. The space domain is $0 < x, y, z < 1$ and time domain is $t > 0$.

$u(x, y, z, 0) = u_0(x, y, z) = sin(\pi x) * cos(\pi y) * cos(\pi z)$

$v(x, y, z, 0) = v_0(x, y, z) = sin(\pi y) * cos(\pi x) * cos(\pi z)$

$w(x, y, z, 0) = w_0(x, y, z) = sin(\pi z) * cos(\pi x) * cos(\pi y)$

as well as:

$u(0, y, z, t) = u(1, y, z, t) = 0$

$v(x, 0, z, t) = v(x, 1, z, t) = 0$

$w(x, y, 0, t) = w(x, y, 1, t) = 0$

## 2. The resolution method

We will build an estimator and have it gradually converge to the 3-tuple solution $U = (u, v, w)$ thanks to a handcrafted loss function based on residuals, computed from original inputs $X = (x, y, z, t)$.

We define:

* A neural model $pinn := U(x, y, z, t)$
* An IC residual function $U0_{residual} := pinn(X, 0) - U0(X)$
* A BC residuals function $Ulim_{residual} := U(0, t) = U(1, t) = 0$
* A PDE residual function $f := U_t + U * U_{.} - \frac{1}{R_e} * U_{..}$

The Physics constraint is a soft-constraint (based on the loss) built by summing the loss of all residuals $L = loss(U0) + loss(Ulim) + loss(f)$.

#### A few of the model's HParams

In [36]:
# number of samples in every dimension
n = 4
grid_shape = (n, n, n, n)

dtype = torch.float

# reynolds number, try for a range of 10^p where p is an integer 
re: float = 100.

# learning rate, classic
lr = 1e-3

#### Helpers

In [37]:
def tuplify(X: torch.Tensor) -> tuple:
    
    x = X[:, 0:1]
    y = X[:, 1:2]
    z = X[:, 2:3]
    t = X[:, 3:4]
    
    return x, y, z, t

def meshify(X: torch.Tensor) -> torch.Tensor:
    x, y, z, t = tuplify(X)
    x, y, z, t = np.meshgrid(x, y, z, t)
    
    x = torch.tensor(x.reshape((-1, 1)))
    y = torch.tensor(y.reshape((-1, 1)))
    z = torch.tensor(z.reshape((-1, 1)))
    t = torch.tensor(t.reshape((-1, 1)))
    
    X = torch.squeeze(torch.stack((x, y, z, t), axis=1))
    
    return X

## 3. The actual implementation

#### a) IC residuals function

Following the article specifications, we'll define the IC with a few cyclical functions.

In [ ]:
def U0(X: torch.Tensor) -> torch.Tensor:
    """Computes the IC as stated previously."""
    
    # X = meshify(X)
    x, y, z, _ = tuplify(X)
    
    u_xyz0 = torch.squeeze(torch.sin(np.pi * x) * torch.cos(np.pi * y) * torch.cos(np.pi * z))
    v_xyz0 = torch.squeeze(torch.sin(np.pi * y) * torch.cos(np.pi * x) * torch.cos(np.pi * z))
    w_xyz0 = torch.squeeze(torch.sin(np.pi * z) * torch.cos(np.pi * x) * torch.cos(np.pi * y))
    
    U0_ = torch.stack((u_xyz0, v_xyz0, w_xyz0), axis=1)
    
    return U0_

In [ ]:
def U0_residuals(X: torch.Tensor) -> torch.Tensor:
    """Computes the residuals for the IC."""
    
    return pinn(X) - U0(X)

#### b) BC residuals function

Residuals on boundary is `0`.

In [40]:
def Ulim_residuals(X: torch.Tensor) -> torch.Tensor:
    """Computes the residuals at the Boundary."""
    
    return pinn(X) - 0.

#### c) PDE residuals function

We need to compute first-order and second-order derivatives of $U$ with respect to $X$. Currently, `torch.__version__ == 1.9.0`, it's a bit tricky, because we cannot filter out *a priori* part of terms that will end-up unused and thus computation is partly wasted. We can only filter *a posteriori*. There's probably some leverage at the DAG level *(Directed Acyclic Graph)*.

PyTorch has a `torch.autograd.functional.hessian()` function but only of output scalars and not vectors so we can't use it.

In [41]:
def f(X: torch.Tensor) -> torch.Tensor:
    """Computes the residuals from the PDE on the rest of the Domain."""
    
    def first_order(X, second_order=False):
        
        U = pinn(X)
        u = torch.squeeze(U[:, 0:1])
        v = torch.squeeze(U[:, 1:2])
        w = torch.squeeze(U[:, 2:3])
        
        U_X = torch.autograd.functional.jacobian(pinn, X, create_graph=True)
        
        u_x = torch.diagonal(torch.squeeze(U_X[:, 0:1, :, 0:1]))
        u_y = torch.diagonal(torch.squeeze(U_X[:, 0:1, :, 1:2]))
        u_z = torch.diagonal(torch.squeeze(U_X[:, 0:1, :, 2:3]))
        u_t = torch.diagonal(torch.squeeze(U_X[:, 0:1, :, 3:4]))
        
        v_x = torch.diagonal(torch.squeeze(U_X[:, 1:2, :, 0:1]))
        v_y = torch.diagonal(torch.squeeze(U_X[:, 1:2, :, 1:2]))
        v_z = torch.diagonal(torch.squeeze(U_X[:, 1:2, :, 2:3]))
        v_t = torch.diagonal(torch.squeeze(U_X[:, 1:2, :, 3:4]))
        
        w_x = torch.diagonal(torch.squeeze(U_X[:, 2:3, :, 0:1]))
        w_y = torch.diagonal(torch.squeeze(U_X[:, 2:3, :, 1:2]))
        w_z = torch.diagonal(torch.squeeze(U_X[:, 2:3, :, 2:3]))
        w_t = torch.diagonal(torch.squeeze(U_X[:, 2:3, :, 3:4]))
        
        if second_order:
            return u, v, w, u_x, u_y, u_z, u_t, v_x, v_y, v_z, v_t, w_x, w_y, w_z, w_t
        
        return u_x, v_y, w_z
    
    # way sub-optimal, the first order jacobian should really be computed once
    # maybe pytorch is doing this lazy, but still, sub-optimal
    def second_order(X):
        U_XX = torch.autograd.functional.jacobian(first_order, X)
        
        u_xx = torch.diagonal(torch.squeeze(U_XX[0][:, :, 0:1]))
        v_xx = torch.diagonal(torch.squeeze(U_XX[1][:, :, 0:1]))
        w_xx = torch.diagonal(torch.squeeze(U_XX[2][:, :, 0:1]))
        
        u_yy = torch.diagonal(torch.squeeze(U_XX[0][:, :, 1:2]))
        v_yy = torch.diagonal(torch.squeeze(U_XX[1][:, :, 1:2]))
        w_yy = torch.diagonal(torch.squeeze(U_XX[2][:, :, 1:2]))
        
        u_zz = torch.diagonal(torch.squeeze(U_XX[0][:, :, 2:3]))
        v_zz = torch.diagonal(torch.squeeze(U_XX[1][:, :, 2:3]))
        w_zz = torch.diagonal(torch.squeeze(U_XX[2][:, :, 2:3]))
        
        return u_xx, u_yy, u_zz, v_xx, v_yy, v_zz, w_xx, w_yy, w_zz
    
    u, v, w, u_x, u_y, u_z, u_t, v_x, v_y, v_z, v_t, w_x, w_y, w_z, w_t = first_order(X, second_order=True)
    u_xx, u_yy, u_zz, v_xx, v_yy, v_zz, w_xx, w_yy, w_zz = second_order(X)
    
    u_ = u_t + u * u_x + v * u_y + w * u_z - re * (u_xx + u_yy + u_zz)
    v_ = v_t + u * v_x + v * v_y + w * v_z - re * (v_xx + v_yy + v_zz)
    w_ = w_t + u * w_x + v * w_y + w * w_z - re * (w_xx + w_yy + w_zz)
    
    U = torch.stack((u_, v_, w_), axis=1)
    
    return U

#### d) The total loss function

Summed-up from all previously defined residuals. Given how input $X$ was produced, it contains both samples from main domain as well as singular values used to compute both IC and BC. We need to carefully route the computation to the right residual function.

In [42]:
def loss(X: torch.Tensor) -> torch.Tensor:
    """Computes the loss based on all residual terms."""
    
    x0 = X[:, 0:1] == 0.
    x1 = X[:, 0:1] == 1.
    xl_ = torch.logical_or(x0, x1)
    xl_ = torch.cat((xl_,) * 4, axis=1)
    xl = torch.masked_select(X, xl_).reshape(-1, 4)
    xl_residuals = torch.mean(torch.square(Ulim_residuals(xl)))
    
    y0 = X[:, 1:2] == 0.
    y1 = X[:, 1:2] == 1.
    yl_ = torch.logical_or(y0, y1)
    yl_ = torch.cat((yl_,) * 4, axis=1)
    yl = torch.masked_select(X, yl_).reshape(-1, 4)
    yl_residuals = torch.mean(torch.square(Ulim_residuals(yl)))
    
    z0 = X[:, 2:3] == 0.
    z1 = X[:, 2:3] == 1.
    zl_ = torch.logical_or(z0, z1)
    zl_ = torch.cat((zl_,) * 4, axis=1)
    zl = torch.masked_select(X, zl_).reshape(-1, 4)
    zl_residuals = torch.mean(torch.square(Ulim_residuals(zl)))
    
    t0_ = X[:, 3:4] == 0.
    t0_ = torch.cat((t0_,) * 4, axis=1)
    t0 = torch.masked_select(X, t0_).reshape(-1, 4)
    t0_residuals = torch.mean(torch.square(U0_residuals(t0)))
    
    or_ = torch.logical_or(t0_, torch.logical_or(zl_, torch.logical_or(xl_, yl_)))
    X_not = torch.logical_not(or_)
    X_ = torch.masked_select(X, X_not).reshape(-1, 4)
    f_residuals = torch.mean(torch.square(f(X_)))
    
    # final loss is simply the sum of residuals
    return torch.mean(torch.stack((
        xl_residuals,
        yl_residuals,
        zl_residuals,
        t0_residuals,
        f_residuals
    )))

#### e) Defining the model

... as a simple straight-forward feed-forward MLP `depth=4` by `width=20` + `activation=nn.Tanh()` defined with PyTorch's sequential API.

In [44]:
# inputs: X = (x, y, z, t)
# outputs: U = (u, v, w)

pinn = nn.Sequential(
    nn.Linear(4, 20, dtype=dtype),
    nn.Tanh(),
    nn.Linear(20, 20, dtype=dtype),
    nn.Tanh(),
    nn.Linear(20, 20, dtype=dtype),
    nn.Tanh(),
    nn.Linear(20, 20, dtype=dtype),
    nn.Tanh(),
    nn.Linear(20, 3, dtype=dtype),
)

## 4. LET'S FIT

Let's start by sampling in both space and time, and create a 4D-meshgrid (main reason why all this is intractable).

In [43]:
x = torch.linspace(0.0, 1.0, steps=n, dtype=dtype).T
y = torch.linspace(0.0, 1.0, steps=n, dtype=dtype).T
z = torch.linspace(0.0, 1.0, steps=n, dtype=dtype).T
t = torch.linspace(0.0, 1.0, steps=n, dtype=dtype).T
X = torch.stack((x, y, z, t), axis=1)

X = meshify(X)

u0 = U0(X)[:, 0:1]
v0 = U0(X)[:, 1:2]
w0 = U0(X)[:, 2:3]

...and loop over epochs... And we're done!

In [45]:
def fit(X: torch.Tensor, 
        epochs: int,
        lr: float = 1e-2):
    """Implements the training loop."""

    optimizer = torch.optim.SGD(pinn.parameters(), lr=lr)
    for epoch in range(epochs):
        
        optimizer.zero_grad()
        loss_ = loss(X)
        loss_.backward()
        optimizer.step()
        
        if epoch % 1000 == 0:
            print(f"epoch: {epoch}, loss: {loss_}")

In [ ]:
fit(X, epochs=10000)

epoch: 0, loss: 0.11308014392852783
epoch: 1000, loss: 0.029296875
epoch: 2000, loss: 0.029296875
epoch: 3000, loss: 0.029296875


**But let's forget about printing anything useful. Simply untractable.**

**For a more realistic and less irrelevant example, checkout `../burger_1d`.**